# Chapter 02: Object-Oriented Programming

## 2.4 Inheritance
In object-oriented programming, the mechanism for a modular and hierarchical organization is a technique known as **inheritance**. This allows a new class to be defined based upon an existing class as the starting point. In object-oriented terminology, the existing class is typically described as the **base class**, **parent class**, or **superclass**, while the newly defined classs is known as the **subclass** or **child class**.

![Classe](../images/Fig2.4.1png.png)


There are two ways in which a subclass can differentiate itself from its superclass. 
- A subclass may **specialize an existing** behavior by providing a new implementation that overrides an existing method. 
- A subclass may also **extend its superclass** by providing brand new methods.

#### Example: Python’s Exception Hierarchy

![Classe](../images/Fig2.5.png)


### 2.4.1 Extending the CreditCard Class

We revisit the CreditCard class of Section 2.3, implementing a subclass  PredatoryCreditCard. The new class will differ from the original in two
ways: 
1. if an attempted charge is rejected because it would have exceeded the credit limit, a $5 fee will be charged, and 

2. there will be a mechanism for assessing a monthly Bank fees  on the outstanding balance, based upon an Annual
Percentage Rate (APR) specified as a constructor parameter.


![Predatory Credit Card](../images/Fig2.6.png)



In [8]:
from ch02.credit_card import CreditCard

class PredatoryCreditCard(CreditCard):
    """An extension to CreditCard that compounds interest and fees."""
    
    def __init__(self, customer, bank, acnt, limit, apr):
        """Create a new predatory credit card instance.
        
        The initial balance is zero.
        customer    the name of the customer
        bank        the name of the bank
        acnt        the acount identifier
        limit       credit limit
        apr         annual percentage rate
        """

        super().__init__(customer, bank, acnt, limit)
        self._apr = apr
        
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed.
        Return False and assess $5 fee if charge is denied.
        """
        
        success = super().charge(price)
        if not success:
            self._balance += 5
        return success
    
    def process_month(self):
        """Assess monthly Bank fees on outstanding balance."""
        if self._balance > 0:
            # if positive balance, convert APR to monthly multiplicative factor
            monthly_factor = pow(1 - self._apr, 1/12)
            self._balance *= monthly_factor

In [9]:
predatory = PredatoryCreditCard('John', 'AA Bank', '1234 5678', 100000000, 0.03)
predatory.charge(1000)
predatory.get_balance()

1000

In [10]:
predatory.process_month()
predatory.get_balance()

997.4649513861633

#### Protected Members
In Python, names beginning with a single underscore are conventionally akin to **protected**, which are accessible to subclasses, but not to the general public, while names beginning with a double underscore (other than special methods) are akin to **private**, which are not accessible to both subclasses and general public.

However, this access control is not supported formally by Python. Keep in mind that it is just a convention. So if you are trying to use some attributes followed by an underscore, think twice that it could compromise the class's designer's intention.

### 2.4.2 Hierarchy of Numeric Progressions
As a second example of use of inheritance, we develop a hierarchy of classes for iterating numeric progressions.

- A numeric progression is a sequence of numbers, where each number depends on one or more of the previous numbers. For example:
    - an arithmetic progression determines the next number by adding a fixed constant to the previous value, and 
    - a geometric progression determines the next number by multiplying the previous value by a fixed constant. 
    
In general, a progression requires a first value, and a way of identifying a new value based on one or more
previous values.

To maximize reusability of code, we develop a hierarchy of classes stemming
from a general base class that we name Progression (s


![Hierarchy of Progression Classes](../images/Fig2.7.png)

In [11]:
class Progression:
    """ITerator producing a generic progression.
    
    Default iterator produces the whole numbers 0, 1, 2, ...
    """
    
    def __init__(self, start=0):
        """Initialize current to the first value of the progression."""
        self._current = start
    
    def _advance(self):
        """Update self._current to a new value.
        
        THis should be overridden by a subclass to customize progression.
        
        By convention, if current is set to None, this designates the end of a progression.
        """
        
        self._current +=1
        
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        if self._current is None:
            raise StopIteration()
        else:
            answer = self._current
            self._advance()
            return answer
        
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self
    
    def print_progression(self, n):
        """Print next n values of the progression."""
        print(' '.join(str(next(self)) for j in range(n)))

#### An Arithmetic Progression Class

In [36]:
class ArithmeticProgression(Progression):
    """Iterator producing an arithmetic progression."""
    
    def __init__(self, increment=1, start=0):
        """Create a new arithmetic progression.
        
        increment   the fixed constant to add to each term (default 1)
        start       the first term of the progression (default 0)
        """
        super().__init__(start)
        self._increment = increment
    
    def _advance(self):
        """Update current value by adding the fixed increment."""
        self._current += self._increment

#### A Geometric Progression Class

In [37]:
class GeometricProgression(Progression):
    """Iterator producing a geometric progression."""
    
    def __init__(self, base=2, start=1):
        """Create a new geometric progression.
        
        base    the fixed constant multiplied to each term (default 2)
        start   the first term of the progression (default 1)
        """
        
        super().__init__(start)
        self._base = base
    
    def _advance(self):
        """Update current value by multiplying it by the base value."""
        self._current *= self._base

#### A Fibonacci Progression Class

In [38]:
class FibonacciProgression(Progression):
    """Iterator producing a generalized Fibonacci progression."""
    
    def __init__(self, first=0, second=1):
        """Create a new fibonacci progression.
        
        first       the first term of the progression (default 0)
        second      the second term of the progression (default 1)
        """
        
        super().__init__(first)
        self._prev = second - first
    
    def _advance(self):
        """Update current value by taking sum of previous two."""
        self._prev, self._current = self._current, self._prev + self._current

#### Testing progressions

In [40]:
Progression().print_progression(10)
ArithmeticProgression(5).print_progression(10)
ArithmeticProgression(5, 2).print_progression(10)
GeometricProgression().print_progression(10)
GeometricProgression(3).print_progression(10)
FibonacciProgression().print_progression(10)
FibonacciProgression(4, 6).print_progression(10)

0 1 2 3 4 5 6 7 8 9
0 5 10 15 20 25 30 35 40 45
2 7 12 17 22 27 32 37 42 47
1 2 4 8 16 32 64 128 256 512
1 3 9 27 81 243 729 2187 6561 19683
0 1 1 2 3 5 8 13 21 34
4 6 10 16 26 42 68 110 178 288


### Abstract Base Class
In classic object-oriented terminology, we say a class is an **abstract base class** if its only purpose is to serve as a base class though inheritance. More formally, an abstract base class is one that cannot be directly instantiated, while **concrete class** is one that can be instantiated. By this definition, our `Progression` class is technically concrete, although we essentially designated it as an abstract base class.

In statically typed languages such as Java and C++, an abstract base class serves as a formal type that may guarantee one or more **abstract methods**. This provides support for polymorphism, as a variable may have an abstract base class as its declared type, even though it refers to an instance of a concrete subclass. Because there are no declared types in Python, this kind of polymorphism can be accomplished without the need for a unifying abstract class. For this reason, there is not as strong a tradition of defining abstract base classes in Python, although Python's `abc` module provides support for defining a formal abstract base class.

Our reason for focusing on abstract base classes in our study of data structures is that Python's `collections` module provides several abstract base classes that assist when defining custom data structures that share a common interface with some of Python's built-in data structures. These rely on an object-oriented software design pattern known as the **template method pattern** The template method pattern is when an abstract base class provides concrete behaviors that rely upon calls to other abstract behaviours. In that way, as soon as a sublclass provides definitions for the missing abstract behaviors, the inherited concrete behaviors are well defined.

As a tangible example, the `collections.Sequence` abstract base class defines behaviors common to Python's `list`, `str`, and `tuple` classes, as sequences that support element access via an integer index. More so, the `collections.Sequence` class provides concrete implementations of methods, `count`, `index` and `__contains__` that can be inherited by any class that provides concrete implementations of both `__len__` and `__getitem__`.

In [41]:
from abc import ABCMeta, abstractmethod


class Sequence(metaclass=ABCMeta):
    """Our own version of collections.Sequence abstract base class."""
    
    @abstractmethod
    def __len__(self):
        """Return the length of the sequence."""
        
    @abstractmethod
    def __getitem__(self, item):
        """Return the element at index item of the sequence."""
    
    def __contains__(self, val):
        """Retrun True if val found in the sequence; False otherwise."""
        for j in range(len(self)):
            if self[j] == val:
                return True
        return False

    def index(self, val):
        """Return leftmost index at which val is found (or raise ValueError)."""
        for j in range(len(self)):
            if self[j] == val:
                return j
        raise ValueError('value not in sequence')
    
    def count(self, val):
        """Return the number of elements equal to given value."""
        k = 0
        for j in range(len(self)):
            if self[j] == val:
                k += 1
            return k

In [44]:
Sequence()

TypeError: Can't instantiate abstract class Sequence with abstract methods __getitem__, __len__

As we can see, since `Sequence` is an abstract Class, not an concrete class, so that it cannot be instantiated.

This implementation relies on two advanced Python techniques. The first is that we declare `ABCMeta` class of the `abc` module as a **metaclass** of our `Sequence` class. *A meta class is different from a superclass*, in that it provides a template for the class definition itself. Specifically, the `ABCMeta` declaration assures that the constructor for the class raises an error.

The second advanced technique is the use of the `@abstractmethod` decorator immediately before the `__len__` and `__getitem__` methods are declared. That declares these two particular methods to be abstract, meaning that we do not provide an implementation within our Sequence base class, but that we expect any concrete subclasses to support those two methods. Python enforces this expectation, by disallowing instantitation for any subclass that does not override the abstract methods with concrete implementations.

We emphasize that if a subclass provides its own implementation of an inherited behaviors from a base class, the new definition overrides the inherited one. This technique can be used when we have the ability to provide a more efficient implementation for a behavior than is achieved by the generic approach. 